In [363]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "impér.": "imperative",
    "impér": "imperative"
}

In [429]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [424]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [364]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


In [659]:
REST = {
    64: "kʺrʹᴇdʹimʹ (creidim) « je crois » ; lɑ̃꞉v χʺlʹe꞉ (lámh chlé) « main gauche »"
}

§ 64.
Exemples de χʺ.
kʺlʹe꞉ (clé) « gauche » ; ɑnəχʺnʹαstə (ana­chneasta) « très poli », de kʺnʹαstə (cneasta) « poli ».

Tous mots que nous tran­scrivons régulière­ment avec χ : də χrʹᴇtʹ ʃeꞏ, etc.

§ 65.
ɑnəǥɑrəv (ana-gharbh) « très rude », de gɑrəv (garbh) « rude » ; mo ǥʌrən (mo dhorn) « mon poing » ; is kᴜmə ǥötʹ (is cuma dhuit) « cela n’a aucune impor­tance pour toi », de dötʹ (duit) « à toi» ; ɑnəǥʷᴇ̈꞉ᵊh (anaghaoth) « vent violent », de gʷᴇ̈꞉h (gaoth) « vent » ; foǥʷɩnʹɩ (fo-dhuine) « quelques personnes », de dɪnʹi (duine) « personne » ; mə ǥrɑ꞉ (mo ghrádh) « mon amour » ; də ǥnɑ꞉h (de ghnáth) « d’habitude ».

§ 66.
Exemples de ǥʺ.
əŋ ǥʺrʹiꞏən ou əŋ ǥrʹiꞏən (an ghrian) « le soleil », de gʺrʹiꞏən (grian) « soleil » ; ɑnəǥʺlʹikʹ (ana-ghlic) « très habile », de gʺlʹikʹ (glic) « habile » ; nə ǥʺnʹi꞉ᵊv (i n‑a ghníomh) « réalisé (en parlant d’une prédic­tion) ».

ɛg (ag) « à » ; əǥʷɪvʹ (agaibh) « à vous », etc.

§ 68.
ç (écrit ch- et parfois sh‑).
ç est une spirante palatale sourde, prononcée plus en avant et avec une ouverture légère­ment plus consi­dérable que le son ich de l’allemand, sans que cependant le phonème cesse d’être nettement spirant et que la confusion avec hʹ soit possible.

ç se rencontre en contact avec des voyelles, ceci seulement en position initiale et, là même, seulement comme alter­nance gramma­ticale de kʹ ou de ʃ suivi de voyelles d’arrière ou mixtes d’arrière ;

dans ce dernier cas ʃ peut déve­lopper un glide palatal j qui, combiné avec le souffle sourd qui repré­sente l’alter­nance initiale de ʃ, donne ç ; là au contraire où ʃ ne développe pas de glide, la palata­lisation ne subsiste pas sous la forme « adoucie » et le résultat de l’aspi­ration est h (cf. § 89). Pour une évolution parallèle de tʹ « adouci », cf. Sommer­felt, Torr, § 197.

ᴀᴜnçαrt (anncheart) « injustice », de kʹαrt (ceart) « droit » ; ɑnəço꞉ (anacheó) « brouil­lard épais », de kʲo꞉ (ceó) « brouil­lard » ; ɑnəçu꞉ⁱnʹ (anachiúin) « très tran­quille », de kʲu꞉ⁱnʹ (ciúin) « tran­quille » : nʹαçαrt (neamh­cheart) « de travers » ; ən vʹαn çì꞉anə (an bhean chéadhna) « la même femme ».

ɩ çɑ꞉ⁱnʹ (a Sheáin) « Jean ! », de ʃʲɑ̃꞉n (Seán) « Jean » ; ɩ ço꞉rʃɩ (a Sheóirse) « Georges ! », de ʃʲo꞉rʃɩ (Seóirse) « Georges » ; ɑnəçᴜ꞉l (ana­shiubhal) « longue marche », de ʃʲᴜ꞉l (siubhal) « marche » ; ço꞉ləmʷɩrʹ (sheól­amair) « nous nous mîmes en marche », de ʃʲo꞉lɩmʹ (seólaim) « je me mets en marche ».

J’ai entendu ç provenant du glide j combiné avec h dans anʹçᴜ (aith­niughadh) « recon­naître ».

§ 69.
j (écrit dh , gh- et quelque­fois non noté).
j est articulé sensible­ment au même point que ç, mais avec une ouverture plus consi­dérable. Ce n’est pas une spirante avec friction nette mais une demi-voyelle compa­rable au y d’anglais yes ou de français yeux.

j apparaît en contact avec des voyelles, en règle générale seulement à l’initiale, le plus souvent comme alter­nance gramma­ticale de dʹ ou gʹ ; on peut noter aussi quelques cas de j appa­raissant à l’initiale devant un i, ou même, en position médiane, à la place d’un i deuxième élément de triph­tongue.

ɑnəjαs (anadheas) « très joli », de dʹαs (deas) « joli » ; nʹiꞏ jinʹɩmʹ (ni dheinim) « je ne fais pas », de dʹᴇnʹɩmʹ (deinim) « je fais » ; mo ji꞉həl (mo dhícheall) « mon possible », de dʹi꞉həl (dícheall) « possible » ; ə χɑrə jilʹ (a chara dhil !) « mon cher ami ! », de dʹilʹ (dil) « cher » ; əm jiꞏɛgʹ (im dhiaidh) « après moi », de dʹiꞏɛgʹ (diaidh) « suite » ; jaᴜl ʃeꞏ (gheall sé) « il promit », de gʹαlɩmʹ (geallaim) « je promets » ; jɑ꞉r ʃeꞏ (ghearr sé) « il coupa », de gʹαrɩmʹ (gearraim) « je coupe ».

dɩ jimʹɩ ʃeꞏ (do imthigh sé) « il s’en alla », de ɩmʹi꞉mʹ (imthighim) « je m’en vais » ; et cf. § 295.

nʹiꞏ vajən (ní bhfaigh­eann) « il n’obtient pas » ; en face de nʹiꞏ vaimʹ (ní bhfaighim) « je n’obtiens pas », cf. § 196.

In [660]:
# DATA[section] = []
section = 64
if not section in DATA:
    DATA[section] = []
extract(REST[section], ";", 2)

4


In [654]:
def split_trans(pos):
    DATA[section][pos]["transcription"] = DATA[section][pos]["transcription"].split(" ou ")

split_trans(-2)

In [662]:
import json

if str(section) in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)

In [661]:
# DATA[section] = DATA[section][0:5]
# del(DATA[section][-1])
DATA[section][-1]["from"] = "clé"

In [639]:
# DATA[section][-2]["french_note"] = DATA[section][-2]["french"]
# DATA[section][-2]["from"] = "crádh"
# del(DATA[section][-2]["french"])
# DATA[section][-2]
DATA[section][-1]["transcription"] = "krɑ꞉"